In [7]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리

In [8]:
pip install selenium

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
from tqdm import tqdm
import calendar
import datetime
import json

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

## 1차 수집

In [ ]:
def get_lyrics(artist, title):
    try:
        url = f'https://api.lyrics.ovh/v1/{artist}/{title}'
        response = requests.get(url)
        return json.loads(response.text)['lyrics']
    except:
        return ''

In [ ]:
songs = pd.read_csv('billboardTop100Totalduplicate_updated.csv', encoding = 'cp949')
songs = songs[['Title', 'Artist']]
songs

In [ ]:
songs['Lyrics'] = ''

In [ ]:
for n in tqdm(range(0, len(songs), 500)):
  start = n
  if n == 5000:
    end = len(songs)
  else:
    end = n + 500
  data = songs.iloc[start:end]
  for i in range(len(data)):
    data.iloc[i, 2] = get_lyrics(data.iloc[i].Artist, data.iloc[i].Title)
    fileName = f'{start}_{end}.csv'
    data.to_csv(fileName)

100%|██████████| 11/11 [22:07<00:00, 120.66s/it]


In [ ]:
df1 = pd.read_csv('data/lyrics/0_500.csv')
df1.

,Unnamed: 0,Title,Artist,Lyrics
0,0,The Monster (feat. Rihanna),Eminem,I'm friends with the monster\r\nThat's under m...
1,1,Timber (feat. Ke$ha),Pitbull,"It's going down, I'm yelling timber\r\nYou bet..."
2,2,Counting Stars,OneRepublic,"Lately, I've been, I've been losing sleep\nDre..."
3,3,Royals,Lorde,I've never seen a diamond in the flesh \nI cut...
4,4,Say Something,"A Great Big World,Christina Aguilera",NaN
...,...,...,...,...
495,495,All About It (feat. Ed Sheeran),Hoodie Allen,Cause I got soul and I won't quit\r\nAnd your ...
496,496,Sugar,Maroon 5,"I'm hurting baby, I'm broken down\r\nI need yo..."
497,497,Heartbeat Song,Kelly Clarkson,This is my heartbeat song and I'm gonna play i...
498,498,Homegrown,Zac Brown Band,"I got a piece of land, out in the countryside\..."


In [ ]:
df1 = pd.read_csv('data/lyrics/0_500.csv').drop(columns = 'Unnamed: 0')
df2 = pd.read_csv('data/lyrics/500_1000.csv').drop(columns = 'Unnamed: 0')
df3 = pd.read_csv('data/lyrics/1000_1500.csv').drop(columns = 'Unnamed: 0')
df4 = pd.read_csv('data/lyrics/1500_2000.csv').drop(columns = 'Unnamed: 0')
df5 = pd.read_csv('data/lyrics/2000_2500.csv').drop(columns = 'Unnamed: 0')
df6 = pd.read_csv('data/lyrics/2500_3000.csv').drop(columns = 'Unnamed: 0')
df7 = pd.read_csv('data/lyrics/3000_3500.csv').drop(columns = 'Unnamed: 0')
df8 = pd.read_csv('data/lyrics/3500_4000.csv').drop(columns = 'Unnamed: 0')
df9 = pd.read_csv('data/lyrics/4000_4500.csv').drop(columns = 'Unnamed: 0')
df10 = pd.read_csv('data/lyrics/4500_5000.csv').drop(columns = 'Unnamed: 0')
df11 = pd.read_csv('data/lyrics/5000_5428.csv').drop(columns = 'Unnamed: 0')

df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11])
df

,Title,Artist,Lyrics
0,The Monster (feat. Rihanna),Eminem,I'm friends with the monster\r\nThat's under m...
1,Timber (feat. Ke$ha),Pitbull,"It's going down, I'm yelling timber\r\nYou bet..."
2,Counting Stars,OneRepublic,"Lately, I've been, I've been losing sleep\nDre..."
3,Royals,Lorde,I've never seen a diamond in the flesh \nI cut...
4,Say Something,"A Great Big World,Christina Aguilera",NaN
...,...,...,...
423,La Victima,Xavi,Ya pensé dos veces antes de que tú \nme la jue...
424,Wondering Why,The Red Clay Strays,NaN
425,DJ Play A Christmas Song,Cher,No one on the streets and the city is quiet\r\...
426,Winter Wonderland,Chloe,NaN


In [ ]:
df.to_csv('lyrics.csv', index = False)

## 2차 수집

In [ ]:
options = webdriver.ChromeOptions( )
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_attribute('headless')
def get_lyrics(keyword):
    driver = webdriver.Chrome(options=options)
    base_url = 'https://www.genie.co.kr/'
    driver.get(base_url)
    search_word = keyword
    # 검색 영역 클릭
    driver.find_element(By.XPATH, '//*[@id="sc-fd"]').click()
    # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="sc-fd"]').send_keys(keyword)
    # 검색 버튼 클릭
    driver.find_element(By.XPATH, '//*[@id="frmGNB"]/fieldset/input[3]').click()
    try:
        # 결과가 있는 경우
        # 곡 정보 버튼 클릭
        driver.find_element(By.XPATH, '//*[@id="body-content"]/div[3]/div[2]/div/table/tbody/tr[1]/td[4]/a').click()
        # 가사 가져오기
        lyrics = driver.find_element(By.XPATH, '//*[@id="pLyrics"]/p').text
        driver.close()
    except:
       # 결과가 없는 경우
       driver.close()
       time.sleep(0.5)
       lyrics = ''
    return lyrics
#songs[0].Lyrics = get_lyrics('98 Braves Morgan Wallen')
songs['Lyrics'] = ''
for i in tqdm(range(len(songs))):
  keyword = songs.iloc[i].Title + ' ' + songs.iloc[i].Artist
  songs.iloc[i, 2] = get_lyrics(keyword)

In [5]:
songs = pd.read_csv('data/lyrics/1차/lyrics.csv')
songs

,Title,Artist,Lyrics
0,The Monster (feat. Rihanna),Eminem,I'm friends with the monster\nThat's under my ...
1,Timber (feat. Ke$ha),Pitbull,"It's going down, I'm yelling timber\nYou bette..."
2,Counting Stars,OneRepublic,"Lately, I've been, I've been losing sleep\nDre..."
3,Royals,Lorde,I've never seen a diamond in the flesh \nI cut...
4,Say Something,"A Great Big World,Christina Aguilera","Say something, I'm giving up on you\r\nI'll be..."
...,...,...,...
5423,La Victima,Xavi,Ya pensé dos veces antes de que tú \nme la jue...
5424,Wondering Why,The Red Clay Strays,NaN
5425,DJ Play A Christmas Song,Cher,No one on the streets and the city is quiet\nI...
5426,Winter Wonderland,Chloe,NaN


In [23]:
empty1 = pd.read_csv('data/lyrics/2차/empty_1.csv').drop(columns = 'Unnamed: 0')
empty2 = pd.read_csv('data/lyrics/2차/empty_2.csv').drop(columns = 'Unnamed: 0')
empty3 = pd.read_csv('data/lyrics/2차/empty_3.csv').drop(columns = 'Unnamed: 0')
empty1

,Title,Artist,Lyrics
0,The Way Life Goes (feat. Nicki Minaj),Lil Uzi Vert,"That's true, that's right (Uzi)\nShe sippin' M..."
1,F**k Love (feat. Trippie Redd),XXXTENTACION,NaN
2,X,Lil Uzi Vert,"Yo, yo Pi'erre,\nyou wanna come out here?\nMet..."
3,For Real,Lil Uzi Vert,NaN
4,UnFazed (feat. The Weeknd),Lil Uzi Vert,NaN
...,...,...,...
1000,Lace It,"Juice WRLD, Eminem,benny blanco",NaN
1001,Mind On You,George Birge,NaN
1002,Wondering Why,The Red Clay Strays,NaN
1003,Winter Wonderland,Chloe,NaN


In [25]:
fill1 = empty1[~((empty1.Lyrics.isna())|(empty1.Lyrics == ''))]
fill2 = empty2[~((empty2.Lyrics.isna())|(empty2.Lyrics == ''))]
fill3 = empty3[~((empty3.Lyrics.isna())|(empty3.Lyrics == ''))]

fill = pd.concat([fill1, fill2, fill3])

In [27]:
fill = fill.drop_duplicates(subset = ['Title', 'Artist'])

In [33]:
fill[(fill.Title == 'The Way Life Goes (feat. Nicki Minaj)')|(fill.Artist == 'Lil Uzi Vert	')].Lyrics.values[0]

"That's true, that's right (Uzi)\nShe sippin' Moet,\nand yeah I swear it get her wetter\nMy Louboutins new,\nso my bottoms, they is redder\nNo I'm not a rat,\nbut I'm all about my cheddar\nJust talked to your homie,\nshe said we should be together\nGave me brain, was so insane\nthat I made her my header\nIf she ever call my phone\nyou know I gotta dead her\nBut I like that girl too much,\nI wish I never met her\n\nNow you beggin' this a doozy\nBut beggars cannot be choosy\nBeen the queen for\neight summers\nJust tell them to call me Suzy\nThis that Nicki, this that Uzi\nThis Hennessy got me woozy\nBroke the internet\nin 5411's and some Gucci\nThe blue ribbons eatin' sushi\nIt's that real good coochie\nThis that wet as Chattahoochee\nFuck all y'all like a hoochie\nGivin' them bars since the teens\nI ain't goin' back to juvie\nI am Nicki Yamaguchi\n'Cause I skate with the ice\nI don't say what's the price\nLookin' like it's a heist\nI been winnin' all my life\nOn a roll, it's a dice\nSin

In [58]:
row = songs.iloc[1].reset_index().melt()
row

,variable,value
0,index,Title
1,index,Artist
2,index,Lyrics
3,1,Timber (feat. Ke$ha)
4,1,Pitbull
5,1,"It's going down, I'm yelling timber\nYou bette..."


In [74]:
def fill_missing_lyrics(row):
  if pd.isna(row['Lyrics']):
    match = fill[(fill['Title'] == row['Title']) & (fill['Artist'] == row['Artist'])]
    if not match.empty:
      return match.iloc[0]['Lyrics']
  return row['Lyrics']

songs['Lyrics'] = songs.apply(fill_missing_lyrics, axis=1)

In [76]:
songs[songs.Lyrics.isna()]

,Title,Artist,Lyrics
1613,F**k Love (feat. Trippie Redd),XXXTENTACION,NaN
1623,For Real,Lil Uzi Vert,NaN
1624,UnFazed (feat. The Weeknd),Lil Uzi Vert,NaN
1680,Patek Water (feat. Offset),"Future,Young Thug",NaN
1682,No Cap,"Future,Young Thug",NaN
...,...,...,...
5421,Lace It,"Juice WRLD, Eminem,benny blanco",NaN
5422,Mind On You,George Birge,NaN
5424,Wondering Why,The Red Clay Strays,NaN
5426,Winter Wonderland,Chloe,NaN


In [77]:
songs.to_csv('lyrics_2.csv')